Docs

In [ ]:
from metaforge.models.metadataentry import MetadataEntry
from metaforge.models.metadatamodel import MetadataModel
from metaforge.parsers.ang_parser import AngParser

import metaforge.utilities.ht_utilities

from pathlib import Path
from typing import List
import hyperthought as ht

## Loading and Existing MetaForge Template ##

We are going to use an existing EBSD 2 Phase template located in the "example_templates" folder and load that into memory using the "from_json_file()" function.

In [ ]:
# Set teh path of the file to store the JSON of the model
ez_file_path = 'example_templates/MultiPhase.ez'

# Read the MetadataModel from the json file
model = MetadataModel.from_json_file(ez_file_path)

## Loading an ANG Dictionary from an existing Data file ##

We now want to load a data file whose values will be extracted and used as the meta data that gets uploaded and tagged onto the uploaded data file. 

In [ ]:
# Set the path of the data file to use to build the MetadataModel
data_prefix = '/Users/Shared/DREAM3D_SDK/'
data_file_path = Path(f'{data_prefix}/DREAM3D_Data/Data/SmallIN100/Slice_1.ang')

# Convert the ANG file into a dictionary
ang_parser = AngParser()
ang_dict = ang_parser.parse_header(data_file_path)

Now we might want to sync the template with the actual ANG data that just got read from the .ang file. This can be important if you want to error out because template values were defined in the template but are *not* available in the actual data file. This can happen for instance if the template extracts data from multiple phases but the data file only has a single phase. In this code we simply print what is missing but proceed on. In real life you would probably want to error check or return early.

In [ ]:
# Sync the the MetadataModel from the ANG header dictionary
missing_entries: List[MetadataEntry] = model.update_model_values_from_dict(ang_dict)
if len(missing_entries) != 0:
  print('Not all values that appear in the Template file were in the input data file.')
  for e in missing_entries:
    print(f'{e.source_path}')

## Interacting with HyperThought ##

Now that we have setup our in memory template and verified our model is correct, we need to authenticate to HyperThought. We do this by creating an **auth_control** variable through the use of the *htauthcontroller* class.

**NOTE TO THE USER**
The access key used here is stale. You will need to go get a new Access Key from your HyperThought website.

In [ ]:
# Use the template to upload to HyperThought
# Set your API Access Key which you would get from HyperThought Web site
accessKey = 'eyJhY2Nlc3NUb2tlbiI6ICI0M2YyMDdiYjVkY2Y0MGMxOGNiNWQ3YjE5NTdmMGE4MiIsICJyZWZyZXNoVG9rZW4iOiAiZGYwMmFlNGU4OTUwNDhmZWJjNTVkYTU3YjJmMTA0YTkiLCAiZXhwaXJlc0luIjogMjg2NiwgImV4cGlyZXNBdCI6ICIyMDIyLTA2LTMwVDExOjQ3OjU0LTA0OjAwIiwgImJhc2VVcmwiOiAiaHR0cHM6Ly9odC5ibHVlcXVhcnR6Lm5ldCIsICJjbGllbnRJZCI6ICIwODc3NjAiLCAiY2xpZW50U2VjcmV0IjogIjJjMzJhYmYyMDBlZGE3MTkxNDQxM2YyYTEwNTE5YmI0YzAzMWZmYjgxOTYwNDQ5OTVlODgxOWVjIn0='

# Create an ht.auth.Authorization to hold the API key, and create WorkspacesAPI and FilesAPI objects
auth_control = ht.auth.Authorization(accessKey, verify=False)
workspaces_api = ht.api.workspaces.WorkspacesAPI(auth_control)
files_api = ht.api.files.FilesAPI(auth_control)

# Upload to a folder created at the root level.
path = ","

# Set the remote directory to create. This DOES NOT check if that folder already exists
remoteDirPath = "Unit_Test"


# Get a list of the projects that the user has access to.
workspaces_list = workspaces_api.get_workspaces()

In [ ]:
# Print the workspace names {Optional}
for workspace in workspaces_list:
    print(f'{workspace["name"]}')

In [ ]:
# Look for a specific workspace by *Workspace Title*
workspace_title = 'Project X-Caliber'
workspace_exists = False
workspace_json = {}
workspace_id = ""
for workspace in workspaces_list:
    if workspace["name"] == workspace_title:
        workspace_exists = True
        workspace_json = workspace
        workspace_id = workspace_json["id"]

# Check to make sure we found the Project. In REAL LIFE you would probably error out at this point if the
# project was not found.
if workspace_exists == False:
    print(f'The requested workspace "{workspace_title}" does not exist')

In [ ]:
# Get the file/folder list from the project listed in the previous code
workspace_folder_list = files_api.get_from_location(space_id=workspace_id,
                                                    path=',',
                                                    file_type=ht.api.files.FilesAPI.FileType.FILES_AND_FOLDERS)

# Print the list of folders/files inside the project of interest [Optional]
for wf in workspace_folder_list:
    print(wf)

In [ ]:
# Set the remote directory to create.
remote_exists = False
remote_folder_name = "Unit_Test"
remote_folder_uuid = ""
print("Checking if remote folder {remote_folder_name} exists")
for f in workspace_folder_list:
    print(f)
    if f["name"] == remote_folder_name:
        remote_exists = True
        remote_folder_uuid = f["pk"]
        print(f'name: {f["name"]}  UUID: {remote_folder_uuid}')
  
if not remote_exists:
    print("Remote Folder does not exist.. creating remote folder {remote_folder_name}")
    remote_folder_uuid = files_api.create_folder(name = remote_folder_name,
                                        space_id = workspace_id,
                                        path = ',',
                                        metadata = None)

    # Get the file/folder list from the project listed in the previous code [Optional]
    # This part is optional but probably necessary in real life to sanity check that the
    # requested directory was created.
    workspace_folder_list = files_api.get_from_location(space_id=workspace_id,
                                                        path=',',
                                                        file_type=ht.api.files.FilesAPI.FileType.FILES_AND_FOLDERS)

    # Print the list of folders/files inside the project of interest [Optional]
    for wf in workspace_folder_list:
        print(wf)

## Uploading Data Files with MetaData ##

We are going to upload a data file with Meta-Data extracted from a given Data file using a give template.

In [ ]:
# Extract the Meta-Data from the Template/Model
missing_entries = []
metadata = ht_utilities.ezmodel_to_ht_metadata(model=model, missing_entries=missing_entries, metadata_file_chosen=True)

# Pick your data files to upload
filelist = f'{data_prefix}DREAM3D_Data/Data/SmallIN100/Slice_1.ang'

# Perform the upload.
remote_folder_id_path = files_api.get_id_path(space_id=workspace_id,
                                              path='/' + remote_folder_name)

file_id, file_name = files_api.upload(local_path=filelist,
                                      space_id=workspace_id,
                                      path=remote_folder_id_path,
                                      metadata=metadata)
print(f'{file_name}: {file_id}')
print("Upload completed")